# Design for Co-Factor Swapping

In [1]:
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization

## Setup Model

In [2]:
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite

In [3]:
# Loading of the model
model = read_sbml_model('data/yeastGEM_het.xml')

In [4]:
with model:
    print(model.slim_optimize())
    model.objective = model.reactions.R07215
    print(model.slim_optimize())

0.08374770604149324
0.04117342484804859


An updated and thus slightly larger model is also available.

In [5]:
model

Name,M_yeastGEM_v8__46__4__46__2
Memory address,0x07fd0342ce2e8
Number of metabolites,2745
Number of reactions,4062
Number of groups,90
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


In [6]:
model.solver = "glpk"

In [7]:
biomass = model.reactions.r_2111

It is probably desirable to account for a growth coupled design. Ideally the value chosen here should correspond to the region chosen in the production envelope.

In [8]:
biomass.lower_bound = 0.1

In [9]:
target = model.metabolites.get_by_id("7-Dehydrocholesterol_c")

In [10]:
demand = model.reactions.get_by_id("DM_7-Dehydrocholesterol_c")

In [11]:
model.objective = demand

In [12]:
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield

In [13]:
prod_yield = product_yield(model.reactions.R07215, model.reactions.r_1714)

By default this method looks to replace NADH and NADPH. You can set different co-factor pairs via the parameter `cofactor_id_swaps=([from], [to])` which requires the metabolite identifiers in the model of the pairs that you want to swap. Please look at the documentation for more info.

In [14]:
CofactorSwapOptimization?

Init signature:
CofactorSwapOptimization(
    cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']),
    candidate_reactions=None,
    skip_reactions=None,
    *args,
    **kwargs,
)
Docstring:     
Optimize co-factor swapping

As suggested in [1]_, flux through a given reaction can sometimes be optimized by swapping complementary
co-factor. This class implements a search for reactions when swapped improve the given objective. Briefly,
the approach is to

- find reactions that have all the targeted co-factor pairs e.g. (nad_c -> nadp_c, nadh_c -> nadph_c)

- add reactions that have the co-factors swapped and then by a search algorithm switching one off in favor of the
  other

The implementation here differs from that in [1]_ in that we use a general purpose search algorithm rather than
formulating the search as a mixed integer linear programming problem.

References
----------
.. [1] King, Zachary A., and Adam M. Feist. "Optimizing Cofactor Specificity of Oxidoreductase Enzym

In [15]:
# Since our model uses other names for the co-factors, here are the names in our model:

# NADH  = s_1203[c]
# NAD+  = s_1198[c]

# NADPH    = s_1212[c]
# NADP+    = s_1207[c]

In [16]:
for metabolite in model.metabolites.query('NADH','name'):
    print(metabolite.id)

s_1203[c]
s_1204[er]
s_1205[m]
s_1206[p]
s_2818[erm]
s_3753[n]


In [17]:
# Swap NADPH to NADH
#                                                                                            cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']),
swap_opt = CofactorSwapOptimization(model=model, objective_function = prod_yield, plot=True, cofactor_id_swaps=(['s_1198[c]', 's_1203[c]'], ['s_1207[c]', 's_1212[c]']))

In [18]:
%%time
result = swap_opt.run(max_size=5)

Starting optimization at Tue, 24 Nov 2020 16:17:11


HBox()

Finished after 00:02:40
CPU times: user 4min 21s, sys: 0 ns, total: 4min 21s
Wall time: 4min 21s


In [19]:
result

,index,targets,fitness


In [20]:
with model:
    print(model.slim_optimize())
    model.objective = model.reactions.R07215
    print(model.slim_optimize())

nan
nan


In [21]:
with model:
    #model.objective = model.reactions.R07215
    sol = model.optimize()
    print(sol.objective_value)

0.0


cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
